# Семинарский ноутбук по компьютерному зрению (Module 5)

В этом задании вам предстоит применить и усовершенствовать методы компьютерного зрения, рассмотренные в лекции, а также провести ряд экспериментов. Ноутбук представляет собой **шаблон**, в котором нужно дописать код и ответы в обозначенных местах.

## Задачи

1. **Чтение и запись видео**: Откройте видеофайл, извлеките метаданные и сохраните копию.
2. **Оптический поток**: Вычислите поток с методом Лукаса-Канаде и визуализируйте.
3. **Распознавание действий**: Классифицируйте действия с R3D_18 и наложите предсказания.
4. **Трекинг объектов**: Отслеживайте объекты с YOLOv8 + ByteTrack и посчитайте метрики.

> **Подсказки**: В каждой секции приведены краткие объяснения, формулы и заготовки функций. Заполняйте `TODO`-блоки и выполняйте эксперименты. Отмечайте свои наблюдения в Markdown-ячейках после выполнения кода.

## Установка и базовые импорты

In [ ]:
import gdown, os, sys, glob, math, time
from pathlib import Path
import cv2, subprocess, shlex, numpy as np
import torch
from torchvision import transforms
from torchvision.models.video import r3d_18, R3D_18_Weights
from ultralytics import YOLO
import matplotlib.pyplot as plt
from IPython.display import HTML

# Скачивание тестового видео
html_video = 'https://..........'
path_video = 'data/task_video.mp4'
video_path = Path(path_video)
video_path.parent.mkdir(parents=True, exist_ok=True)
if not video_path.exists():
    gdown.download(html_video, str(video_path), quiet=False)

OUT_DIR = Path("out"); OUT_DIR.mkdir(exist_ok=True, parents=True)

def list_videos(vdir: Path):
    vids = sorted([p for ext in ("*.mp4", "*.avi", "*.mov", "*.mkv") for p in vdir.glob(ext)])
    if not vids:
        raise FileNotFoundError(f"В {vdir} не найдено видеофайлов.")
    return vids

videos = [video_path] if video_path.exists() and video_path.is_file() else list_videos(video_path.parent)
SRC = str(videos[0])

def html_video_autosize(path: str, width: int = 512):
    p = Path(path)
    assert p.exists(), f"Файл не найден: {p}"
    cap = cv2.VideoCapture(str(p))
    ok, frame = cap.read()
    cap.release()
    assert ok, f"Не удалось прочитать кадр из: {p}"
    h, w = frame.shape[:2]
    new_h = int(h * (width / w))
    return HTML(f'<video src="{p}" width="{width}" height="{new_h}" controls></video>')

## 1. Чтение и запись видео

Базовая операция с видео включает открытие потока, извлечение кадров и сохранение результата. Это фундамент для дальнейшей обработки.

### Что нужно сделать
- Откройте видеофайл с использованием `cv2.VideoCapture`.
- Извлеките метаданные: FPS, разрешение, количество кадров.
- Сохраните видео с копированием оригинальных кадров через ffmpeg.
- Отобразите результат с помощью HTML.

In [ ]:
# TODO: Чтение и запись видео
def open_h264_writer(out_path, width, height, fps, crf=20, preset="veryfast"):
    cmd = [
        'ffmpeg', '-y', '-f', 'rawvideo', '-vcodec', 'rawvideo',
        '-s', f'{width}x{height}', '-pix_fmt', 'bgr24', '-r', str(fps),
        '-i', '-', '-an', '-vcodec', 'libx264', '-crf', str(crf),
        '-preset', preset, '-pix_fmt', 'yuv420p', str(out_path)
    ]
    return subprocess.Popen(cmd, stdin=subprocess.PIPE)

# TODO: Откройте видео и извлеките метаданные
cap = cv2.VideoCapture(SRC)
assert cap.isOpened(), f"Не открыть {SRC}"
fps = # TODO
w = # TODO
h = # TODO
total_frames = # TODO
print(f"FPS: {fps}, Resolution: {w}x{h}, Total Frames: {total_frames}")

OUT_MP4 = OUT_DIR / "copy_h264.mp4"
# TODO: Создайте writer и запишите кадры
writer = open_h264_writer(OUT_MP4, w, h, fps)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    # TODO: Запишите кадр

cap.release()
writer.stdin.close()
writer.wait()

html_video_autosize(OUT_MP4)

## 2. Оптический поток

Оптический поток (Optical Flow) описывает движение объектов между кадрами. Метод Лукаса-Канаде вычисляет смещение пикселей, предполагая локальную константность движения.

Формулы для градиентов:
$$I_x = \frac{\partial I}{\partial x}, \quad I_y = \frac{\partial I}{\partial y}, \quad I_t = \frac{\partial I}{\partial t}$$
$$Au = b, \quad u = (A^TA)^{-1} A^T b$$

### Что нужно сделать
- Вычислите оптический поток с помощью `cv2.calcOpticalFlowPyrLK`.
- Добавьте повторное обнаружение точек, если их мало, для стабильности.
- Визуализируйте векторы движения стрелками на кадрах.
- Сохраните видео с оверлеем.

In [ ]:
# TODO: Оптический поток
cap = cv2.VideoCapture(SRC)
assert cap.isOpened(), f"Не открыть {SRC}"
fps = cap.get(cv2.CAP_PROP_FPS) or 25
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

OUT_MP4 = OUT_DIR / "optical_flow_lk_h264.mp4"
writer = open_h264_writer(OUT_MP4, w, h, fps)

ret, prev_frame = cap.read()
assert ret, "Не удалось прочитать первый кадр"
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
prev_pts = cv2.goodFeaturesToTrack(prev_gray, maxCorners=200, qualityLevel=0.01, minDistance=30)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # TODO: Вычислите оптический поток
    next_pts, status, err = # TODO
    good_new = next_pts[status == 1]
    good_old = prev_pts[status == 1]

    # TODO: Визуализируйте векторы движения
    for new, old in zip(good_new, good_old):
        a, b = new.ravel()
        c, d = old.ravel()
        cv2.arrowedLine(frame, (int(c), int(d)), (int(a), int(b)), (0, 255, 0), 2)
    
    writer.stdin.write(frame.tobytes())
    prev_gray = gray.copy()
    prev_pts = good_new.reshape(-1, 1, 2)
    
    # TODO: Повторное обнаружение точек, если их мало
    if len(prev_pts) < 50:
        new_pts = # TODO
        if new_pts is not None:
            prev_pts = np.vstack((prev_pts, new_pts))

cap.release()
writer.stdin.close()
writer.wait()

html_video_autosize(OUT_MP4)

## 3. Распознавание действий

Распознавание действий использует 3D-свёрточные сети (например, R3D_18), обученные на датасете Kinetics-400, для классификации действий в видеоклипах.

### Что нужно сделать
- Загрузите предобученную модель R3D_18.
- Извлеките клипы по 16 кадров, преобразуйте и классифицируйте.
- Визуализируйте топ-1 предсказание.
- Наложите предсказания на кадры.

In [ ]:
# TODO: Распознавание действий
cap = cv2.VideoCapture(SRC)
assert cap.isOpened(), f"Не открыть {SRC}"
fps = cap.get(cv2.CAP_PROP_FPS) or 25
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

OUT_MP4 = OUT_DIR / "action_recognition_overlay_h264.mp4"
writer = open_h264_writer(OUT_MP4, w, h, fps)

weights = R3D_18_Weights.DEFAULT
model = # TODO
model.eval()

preprocess = transforms.Compose([
    # TODO: Преобразования (ToTensor, Resize, Normalize)
])

clip = []
pred_label = ""
NUM_FRAMES = 16

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pil_image = Image.fromarray(frame_rgb)
    processed_frame = # TODO
    clip.append(processed_frame)
    if len(clip) == NUM_FRAMES:
        clip_tensor = # TODO
        input_tensor = # TODO
        with torch.no_grad():
            output = # TODO
        top1 = torch.topk(output, 1)
        pred_label = weights.meta["categories"][top1.indices[0][0]]
        clip = clip[1:]
    cv2.putText(frame, pred_label if pred_label else "Analyzing...", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    writer.stdin.write(frame.tobytes())

cap.release()
writer.stdin.close()
writer.wait()

html_video_autosize(OUT_MP4)

## 4. Трекинг объектов

Трекинг объектов использует YOLOv8 для детекции и ByteTrack для ассоциации объектов между кадрами. Добавлены метрики: скорость и время в зоне интереса.

### Что нужно сделать
- Настройте модель YOLOv8 для детекции объектов (расширьте на все классы).
- Вычислите траектории, скорости и время в ROI.
- Визуализируйте bbox, ID и метрики.
- Отобразите сводку статистики.

In [ ]:
# TODO: Трекинг объектов
def bbox_center_xyxy(box):
    # TODO: Вычислите центр bbox

def in_zone(point, box):
    # TODO: Динамическая зона вокруг объекта

def draw_metrics_label(frame, text_lines, anchor, scale=1.0):
    # TODO: Отрисовка метрик

def update_stats(obj_id, center, inroi, step_dist):
    # TODO: Обновление статистики

cap = cv2.VideoCapture(SRC)
assert cap.isOpened(), f"Не открыть {SRC}"
fps = cap.get(cv2.CAP_PROP_FPS) or 25
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

OUT_MP4 = OUT_DIR / "metrics_yolo_bytetrack_overlay_h264.mp4"
writer = open_h264_writer(OUT_MP4, w, h, fps)

yolo = # TODO: Загрузите модель
stats = {}

for r in yolo.track(source=SRC, tracker="bytetrack.yaml", conf=0.1, iou=0.5, stream=True, verbose=False):
    frame = r.orig_img.copy()
    H, W = frame.shape[:2]
    SCALE = min(max(W, H) / 1920.0, 1.9)

    boxes_xyxy = # TODO: Извлеките bbox
    ids = # TODO: Извлеките ID
    clses = # TODO: Извлеките классы
    confs = # TODO: Извлеките уверенность

    for b, oid, cls, cf in zip(boxes_xyxy, ids, clses, confs):
        # TODO: Обработка объектов (расширьте на все классы)
        cx, cy = bbox_center_xyxy(b)
        # TODO: Вычислите скорость и зону
        # TODO: Обновите статистику
        # TODO: Отрисуйте bbox и метрики

    writer.stdin.write(frame.tobytes())

writer.stdin.close()
writer.wait()

# TODO: Вывод сводки
summary = []
for oid, st in stats.items():
    # TODO: Рассчитайте метрики

summary.sort(key=lambda x: x[0])
print("\nID | avg_speed [px/s] | time_in_zone [s] | tracked_time [s]")
for oid, v, tin, tt in summary:
    print(f"{oid:2d} | {v:14.1f} | {tin:15.2f} | {tt:14.2f}")
print(f"\nВидео с оверлеем сохранено: {OUT_MP4}")

html_video_autosize(OUT_MP4)

## Выводы и обсуждение

После выполнения всех заданий сформулируйте краткие выводы:

* Как параметры влияют на FPS и качество видео?
* Как оптический поток справляется с быстрым движением?
* Какие действия предсказывает модель R3D_18?
* Как метрики трекинга помогают в анализе?

Запишите свои наблюдения и выводы.
